In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

In [3]:
from workflow_widgets import WorkflowWidget
from droplet_workflow import droplet_wf
import workflow_objects as kale

In [4]:
from concurrent.futures import ThreadPoolExecutor

In [5]:
import fireworks as fw
from fireworks.core.rocket_launcher import rapidfire

In [6]:
from aux_widgets import Space

In [7]:
import ipywidgets as ipw

In [8]:
from IPython import display as disp

In [9]:
ipw.interact(lambda x: x, x=True)

A Jupyter Widget

<function __main__.<lambda>>

# Example Workflow

## Define Workflow

In [10]:
t = kale.Task('test')

example_wf = kale.Workflow('example_wf')

n_echos = 10
n_tasks = 3
out_file = "out.txt"

pwd_task = kale.CommandLineTask(
    name='pwd_task',
    command="""echo "Hello from '`pwd`'." > {out_file}""",
    output_files=["{out_file}"],
    params=dict(out_file=out_file)
)

echo_tasks = [
    kale.CommandLineTask(
        name='echo_task_{i}',
        command='for j in {{1..{n_echos}}}; do echo "Hello #{i}: $j" >> {out_file}; sleep 1; done',
        output_files=["{out_file}"],
        params=dict(out_file=out_file, i=i, n_echos=n_echos)
    )
    for i in range(n_tasks)
]

done_task = kale.CommandLineTask(
    name='done_task',
    command="echo 'Done!' >> {out_file}",
    output_files=["{out_file}"],
    params=dict(out_file=out_file)
)

example_wf.add_task(pwd_task)
for i in range(n_tasks):
    example_wf.add_task(echo_tasks[i], dependencies=[pwd_task])
example_wf.add_task(done_task, dependencies=echo_tasks)


In [11]:
%cd /home/oliver/lbl/jupyter_hpc

pool = kale.WorkerPool(num_workers=4)

pool.log_area.clear_output()
pool.log_area

/home/oliver/lbl/jupyter_hpc


ServerSelectionTimeoutError: localhost:27017: [Errno 111] Connection refused

In [13]:
w = WorkflowWidget(example_wf)

In [14]:
w

A Jupyter Widget

In [99]:
w.workflow.

In [15]:
pool.fw_run(example_wf)

2017-08-12 15:19:39,303 INFO Added a workflow. id_map: {-5: 1, -4: 2, -3: 3, -2: 4, -1: 5}
2017-08-12 15:19:49,816 INFO Task completed: ScriptTask 
2017-08-12 15:19:49,851 INFO Rocket finished
2017-08-12 15:19:50,411 INFO Task completed: ScriptTask 
2017-08-12 15:19:50,443 INFO Rocket finished
2017-08-12 15:19:51,414 INFO Task completed: ScriptTask 
2017-08-12 15:19:51,447 INFO Rocket finished
2017-08-12 15:19:51,451 INFO Created new dir /home/oliver/lbl/jupyter_hpc/launcher_2017-08-12-22-19-51-451754
2017-08-12 15:19:51,453 INFO Launching Rocket
2017-08-12 15:19:51,479 INFO RUNNING fw_id: 1 in directory: /home/oliver/lbl/jupyter_hpc
2017-08-12 15:19:51,483 INFO Task started: ScriptTask.
2017-08-12 15:19:51,498 INFO Task completed: ScriptTask 
2017-08-12 15:19:51,521 INFO Rocket finished


# I need a widget to manage/create WorkerPools.

WorkflowWidget should take WorkerPools as arguments. Or at least contain a dropdown menu to select them.

OR: We could separate the running and viewing altogether. So there would be another widget with any instantiated workflows and UI to associate them with WorkerPools and run them. I like this the best, I think.

# Table needs:
- add after row
- remove row

In [44]:
class Table(ipw.VBox):
    def __init__(self, contents, align='left', width=600):
        "Contents should be a rectangular list of lists or numpy array"
        super().__init__()
        
        if type(width) is int:
            self.width_u = u'{}px'.format(width)
            self.width_int = width
        else:
            try:
                if '%' in width:
                    raise NotImplementedError("Specify absolute width, not %.")
            except TypeError:
                pass
        
            self.width_u = u'{}'.format(width)
            self.width_int = int(width.strip('px'))
            
        self.num_cols = max([len(row) for row in contents])
        try:
            self.cell_width_int = self.width_int // self.num_cols
        except ZeroDivisionError:
            self.cell_width_int = 0
        self.cell_width_u = u'{}px'.format(self.cell_width_int)
        
        # Row alignment
        if align == 'left':
            flex_align = 'flex-start'
        elif align == 'right':
            flex_align = 'flex-end'
        else:
            flex_align = align
        
        self.layout = ipw.Layout(
            width=self.width_u
        )
        
        self.row_layout = ipw.Layout(
            width=self.width_u,
            justify_content='space-between'
        )
        self.cell_layout = ipw.Layout(
            width=self.cell_width_u,
            justify_content=flex_align
        )
        
        self.from_lists(contents)
        
    def from_lists(self, contents):
        "Create table from lists"
        for row in contents:
            self.add_row(row)
                
    def add_row(self, row):
        "Append row to end of table"
        row_box = self.parse_row(row)
        self.children += (row_box,)
    
    def parse_row(self, row):
        "Create usable widgets from row contents"
        cell_boxes = []
        for cell in row:
            if type(cell) is str:
                cell_widget = ipw.HTML(cell)
            else:
                cell_widget = cell
                
            cell_boxes.append(
                ipw.HBox(
                    [cell_widget],
                    layout=self.cell_layout,
                )
            )
        
        row_box = ipw.HBox(
            cell_boxes,
            layout=self.row_layout
        )
        
        return row_box
    
    def insert_row(self, index, row):
        "Insert `row` at before row `index`"
        row_box = self.parse_row(row)
        new_children = list(self.children)
        new_children.insert(index, row_box)
        self.children = new_children

In [45]:
t = Table([
    [ipw.Button(), ipw.IntSlider()],
    [ipw.Text(), ipw.IntSlider()],
    ["Test","123"]
],width=600,align='right')
t

A Jupyter Widget

In [50]:
t.add_row(["#5", ipw.IntSlider(value=6)])

In [14]:
t.children[0].children[0].layout.width

'300px'

In [56]:
class WorkerPoolWidgetVBox(ipw.VBox):
    def __init__(self):
        
        super().__init__()
        
        # UI
        self._pool_table = Table([[
            "Name", "Status", "# of Workers", "Action"
        ]])
        
        self._new_button = ipw.Button(
            icon="plus",
            button_style="success",
        )
        
        self._name_text = ipw.Text(description="name")
        self._num_workers_text = ipw.IntText(description="# of workers",value=1)
        
        self.children = [
            self._pool_table,
            ipw.HBox([
                self._name_text,
                self._num_workers_text,
                self._new_button
            ])
        ]
        
        # Logic
        self.pool_list = []
        self._new_button.on_click(self.add_pool)
    
    def add_pool(self, caller):
        num_workers = self._num_workers_text.value 
        name = self._name_text.value
        
        self.pool_list.append(kale.WorkerPool(num_workers))
        self._pool_table.add_row(
            [name, "Running", str(num_workers), "N/A"]
        )


In [58]:
class WorkerPoolWidget(Table):
    def __init__(self):
        
        # UI
        self._name_text = ipw.Text()
        self._num_workers_text = ipw.IntText(value=1)
        self._new_button = ipw.Button(
            icon="plus",
            button_style="success",
        )
        
        super().__init__([
            ["Name", "Status", "# of Workers", "Action"],
            [self._name_text, "", self._num_workers_text, self._new_button]
        ], width=800)
        
        # Logic
        self.pool_list = []
        self._new_button.on_click(self.add_pool)
    
    def add_pool(self, caller):
        num_workers = self._num_workers_text.value 
        name = self._name_text.value
        
        self.pool_list.append(kale.WorkerPool(num_workers))
        self.insert_row(
            -1,
            [name, "Running", str(num_workers), "N/A"]
        )


# This widget (and WorkerPools) need(s):
- Remove buttons
- Deactivate WorkerPool?
- Stop/Run?
- Better styling for table like 

In [ ]:
w.add_pool()

In [ ]:
w = WorkerPoolWidget()
w

In [307]:
l = t.children[0].layout

In [309]:
l.align_content

In [284]:
t.children

()

In [81]:
with out:
    disp.display(ipw.df)

In [ ]:
html_output = 

with out
    disp.display(
        disp.HTML(
            html_output
        )
    )

---

# Droplet Workflow

In [ ]:
WorkflowWidget(droplet_wf)

In [ ]:
import bqplot as bq

In [ ]:
bq.Figure()

In [ ]:
from bqplot.marks import Graph

In [ ]:
ipw.Accordion?

In [ ]:
ipw.Accordion(children=[ipw.IntSlider()])